In [1]:
import csv
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict, Counter
from dateutil.parser import *




def nationalraw(fn="nationalraw.csv"):

    varmap = {}
    with open("variable_mapping.json", "r") as fp:
        varmap = json.load(fp)
  
    date = ""
    data = {}
    with open(fn, "r") as fp:
        reader = csv.reader(fp)
        header = next(reader)
        for row in reader:
            d = {k:v.strip() for k, v in zip(header, row)}
            d_new = {}
            for k in varmap.keys():
                if (k == "region_text") | (k == "urbanrural_text"):
                    d_new[k] = d[k]
                    continue
                try:
                    d_new[k] = float(d[k])
                except ValueError:
                    pass
            k = ""
            if d["county"] != "":
                k = d["state"].zfill(2) + d["county"].zfill(3)
            elif d["state"] != "":
                k = d["state"].zfill(2)
            else:
                k = "_nation"
            data[k] = d_new 
            if date < d["date"]:
                date = d["date"]

    with open("../data.json", "w") as fp:
        json.dump(data, fp)

    with open("../date.json", "w") as fp:
        json.dump({"date": date}, fp, indent=2)

if __name__=="__main__":

    nationalraw()


In [1]:
import csv
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict, Counter
from dateutil.parser import *
nationalraw = pd.read_csv("nationalraw.csv", dtype = {'state': str, 'county': str, 'countynum': str})

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
import plotly.express as px

colorscale = ["#e1dce2", "#d3b6cd","#bf88b5", "#af5194", "#99528c", "#633c70", "#ffffff"]

from PIL import Image 
left = 100
top = 50
right = 1000
bottom = 800
# len(nationalraw[nationalraw["county"].notna()])
# np.isnan(nationalraw["county"][0])
# pd.isnull(nationalraw["county"][0]) == False
# len(nationalraw["countynum"][2]) ==4

In [2]:
# !pip install --upgrade plotly
# !pip install --upgrade geopandas
# !pip install --upgrade pyshp
# !pip install --upgrade shapely
# !conda install -c plotly plotly-geo
# !conda install -c plotly plotly-orca
# !conda install us

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - us

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [22]:
%%time
for i in range(len(nationalraw)):
    if (pd.isnull(nationalraw["countynum"][i]) == False):
        if (len(nationalraw["countynum"][i]) == 4):
            nationalraw["countynum"][i] = "0" + nationalraw["countynum"][i]
        

/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



CPU times: user 8min 10s, sys: 1.69 s, total: 8min 12s
Wall time: 8min 17s


In [23]:
all_counties = nationalraw[nationalraw["countynum"].notna()]

In [8]:
##ccvi

all_counties['quintile'] = all_counties['ccvi'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

# all_counties.loc[all_counties.quintile == "1.0", 'quintile'] = endpts[0]
# all_counties.loc[all_counties.quintile == "2.0", 'quintile'] = endpts[1]
# all_counties.loc[all_counties.quintile == "3.0", 'quintile'] = endpts[2]
# all_counties.loc[all_counties.quintile == "4.0", 'quintile'] = endpts[3]
# all_counties.loc[all_counties.quintile == "5.0", 'quintile'] = endpts[4]
# all_counties.loc[all_counties.quintile == "6.0", 'quintile'] = endpts[5]
# all_counties.loc[all_counties.quintile == "nan", 'quintile'] = "Not Reported"

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/ccvi.png")

img = Image.open(r"../../NationalReportImages/ccvi.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/ccvi.png")

/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [24]:
##poverty
all_counties['quintile'] = all_counties['poverty'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/poverty.png")

img = Image.open(r"../../NationalReportImages/poverty.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/poverty.png")

/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [10]:
##black
all_counties['quintile'] = all_counties['black'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/black.png")

img = Image.open(r"../../NationalReportImages/black.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/black.png")

In [11]:
##resSeg
all_counties['quintile'] = all_counties['resSeg'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/resSeg.png")

img = Image.open(r"../../NationalReportImages/resSeg.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/resSeg.png")

In [12]:
##anycondition
all_counties['quintile'] = all_counties['anycondition'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/anycondition.png")

img = Image.open(r"../../NationalReportImages/anycondition.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/anycondition.png")

In [13]:
##copd
all_counties['quintile'] = all_counties['copd'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/copd.png")

img = Image.open(r"../../NationalReportImages/copd.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/copd.png")

In [14]:
##ckd
all_counties['quintile'] = all_counties['ckd'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/ckd.png")

img = Image.open(r"../../NationalReportImages/ckd.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/ckd.png")

In [15]:
##heartdisease
all_counties['quintile'] = all_counties['heartdisease'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/heartdisease.png")

img = Image.open(r"../../NationalReportImages/heartdisease.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/heartdisease.png")

In [16]:
##diabetes
all_counties['quintile'] = all_counties['diabetes'].transform(
                     lambda x: pd.qcut(x, 6, labels=range(1,7)))
all_counties = all_counties.sort_values(by = "quintile")
all_counties["quintile"] = all_counties["quintile"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='quintile',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/diabetes.png")

img = Image.open(r"../../NationalReportImages/diabetes.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/diabetes.png")

In [22]:
##urbanrural
all_counties = all_counties.sort_values(by = "urbanrural")
all_counties["urbanrural"] = all_counties["urbanrural"].astype(str)

fig = px.choropleth(all_counties, geojson=counties, locations='countynum', color='urbanrural',
                    color_discrete_sequence=colorscale,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/urbanrural.png")

img = Image.open(r"../../NationalReportImages/urbanrural.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/urbanrural.png")

In [9]:
%%time
nationalraw['state'] = nationalraw['state'].apply(lambda x: str(x).zfill(2))

CPU times: user 78 ms, sys: 25.4 ms, total: 103 ms
Wall time: 106 ms


In [17]:
all_state = nationalraw[nationalraw["state"] != "nan"]
abb_list = {
            '01': {
                'state_cd': '01',
                'state_abbr': 'AL'
            },
            '02': {
                'state_cd': '02',
                'state_abbr': 'AK'
            },
            '04': {
                'state_cd': '04',
                'state_abbr': 'AZ'
            },
            '05': {
                'state_cd': '05',
                'state_abbr': 'AR'
            },
            '06': {
                'state_cd': '06',
                'state_abbr': 'CA'
            },
            '08': {
                'state_cd': '08',
                'state_abbr': 'CO'
            },
            '09': {
                'state_cd': '09',
                'state_abbr': 'CT'
            },
            '10': {
                'state_cd': '10',
                'state_abbr': 'DE'
            },
            '11': {
                'state_cd': '11',
                'state_abbr': 'DC'
            },
            '12': {
                'state_cd': '12',
                'state_abbr': 'FL'
            },
            '13': {
                'state_cd': '13',
                'state_abbr': 'GA'
            },
            '15': {
                'state_cd': '15',
                'state_abbr': 'HI'
            },
            '16': {
                'state_cd': '16',
                'state_abbr': 'ID'
            },
            '17': {
                'state_cd': '17',
                'state_abbr': 'IL'
            },
            '18': {
                'state_cd': '18',
                'state_abbr': 'IN'
            },
            '19': {
                'state_cd': '19',
                'state_abbr': 'IA'
            },
            '20': {
                'state_cd': '20',
                'state_abbr': 'KS'
            },
            '21': {
                'state_cd': '21',
                'state_abbr': 'KY'
            },
            '22': {
                'state_cd': '22',
                'state_abbr': 'LA'
            },
            '23': {
                'state_cd': '23',
                'state_abbr': 'ME'
            },
            '24': {
                'state_cd': '24',
                'state_abbr': 'MD'
            },
            '25': {
                'state_cd': '25',
                'state_abbr': 'MA'
            },
            '26': {
                'state_cd': '26',
                'state_abbr': 'MI'
            },
            '27': {
                'state_cd': '27',
                'state_abbr': 'MN'
            },
            '28': {
                'state_cd': '28',
                'state_abbr': 'MS'
            },
            '29': {
                'state_cd': '29',
                'state_abbr': 'MO'
            },
            '30': {
                'state_cd': '30',
                'state_abbr': 'MT'
            },
            '31': {
                'state_cd': '31',
                'state_abbr': 'NE'
            },
            '32': {
                'state_cd': '32',
                'state_abbr': 'NV'
            },
            '33': {
                'state_cd': '33',
                'state_abbr': 'NH'
            },
            '34': {
                'state_cd': '34',
                'state_abbr': 'NJ'
            },
            '35': {
                'state_cd': '35',
                'state_abbr': 'NM'
            },
            '36': {
                'state_cd': '36',
                'state_abbr': 'NY'
            },
            '37': {
                'state_cd': '37',
                'state_abbr': 'NC'
            },
            '38': {
                'state_cd': '38',
                'state_abbr': 'ND'
            },
            '39': {
                'state_cd': '39',
                'state_abbr': 'OH'
            },
            '40': {
                'state_cd': '40',
                'state_abbr': 'OK'
            },
            '41': {
                'state_cd': '41',
                'state_abbr': 'OR'
            },
            '42': {
                'state_cd': '42',
                'state_abbr': 'PA'
            },
            '44': {
                'state_cd': '44',
                'state_abbr': 'RI'
            },
            '45': {
                'state_cd': '45',
                'state_abbr': 'SC'
            },
            '46': {
                'state_cd': '46',
                'state_abbr': 'SD'
            },
            '47': {
                'state_cd': '47',
                'state_abbr': 'TN'
            },
            '48': {
                'state_cd': '48',
                'state_abbr': 'TX'
            },
            '49': {
                'state_cd': '49',
                'state_abbr': 'UT'
            },
            '50': {
                'state_cd': '50',
                'state_abbr': 'VT'
            },
            '51': {
                'state_cd': '51',
                'state_abbr': 'VA'
            },
            '53': {
                'state_cd': '53',
                'state_abbr': 'WA'
            },
            '54': {
                'state_cd': '54',
                'state_abbr': 'WV'
            },
            '55': {
                'state_cd': '55',
                'state_abbr': 'WI'
            },
            '56': {
                'state_cd': '56',
                'state_abbr': 'WY'
            },
            '60': {
                'state_cd': '60',
                'state_abbr': 'AS'
            },
            '64': {
                'state_cd': '64',
                'state_abbr': 'FM     '
            },
            '66': {
                'state_cd': '66',
                'state_abbr': 'GU'
            },
            '68': {
                'state_cd': '68',
                'state_abbr': 'MH'
            },
            '69': {
                'state_cd': '69',
                'state_abbr': 'MP'
            },
            '70': {
                'state_cd': '70',
                'state_abbr': 'PW'
            },
            '72': {
                'state_cd': '72',
                'state_abbr': 'PR'
            },
            '74': {
                'state_cd': '74',
                'state_abbr': 'UM'
            },
            '78': {
                'state_cd': '78',
                'state_abbr': 'VI'
            }
        }
all_state['abb'] = all_state['state'].apply(lambda x: abb_list[x]['state_abbr'])


/Applications/Anaconda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
##region
colorscaleRegion = ["#bf88b5", "#af5194", "#99528c", "#633c70"]

all_state = all_state.sort_values(by = "region")
all_state["region"] = all_state["region"].astype(str)

fig = px.choropleth(all_state, locations='abb', locationmode="USA-states", color='region',
                    color_discrete_sequence=colorscaleRegion,
                           scope="usa"
                          )
fig.update_layout(height=892, width =1200, margin={"r":100,"t":100,"l":100,"b":100}, 
                  geo = dict(
                    showlakes=True, # lakes
                    lakecolor='rgb(255, 255, 255)'))
fig.update(layout_showlegend=False)
fig.update_traces(marker_line_width=0.3)
# fig.show()
fig.write_image("../../NationalReportImages/region.png")

img = Image.open(r"../../NationalReportImages/region.png") 
img_res = img.crop((left, top, right, bottom)) 
img_res.save("../../NationalReportImages/region.png")

In [1]:
# all_counties['quintile'] = pd.qcut(all_counties['poverty'], 6)
# all_counties['quintile']
# all_counties["quintile"] = all_counties["quintile"].astype(str)

# endpts = list(np.mgrid[min(all_counties['poverty']):max(all_counties['poverty']):6j])

# all_counties['quintile']


In [2]:
# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#     counties = json.load(response)

# import pandas as pd

# import plotly.graph_objects as go

# fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=all_counties['countynum'], z=all_counties['poverty'],
                                    
#                                     marker_opacity=0.5, marker_line_width=0))
# fig.update_layout(mapbox_style="carto-positron",
#                   mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()